## Предобработка и разметка текста

In [1]:
!pip install pymorphy2
!git clone https://github.com/nlpub/pymystem3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 884.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5268037afcb310bece4f23faa9286d0d9b09953f8f68f1844d94994cc9f74d13
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Cloning into 'pymystem3'...
remote: Enumerating objects: 434, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 434 (delta 4), reused 0 (delta 0), pack-reused 425
Receiving objects: 100% (434/434), 100.19 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [2]:
import pandas as pd
import re

import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
nltk.download('punkt')

from nltk.tokenize import word_tokenize
import pymorphy2
from pymorphy2 import MorphAnalyzer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
path = '/content/drive/MyDrive/Магистратура/Структурирование и разметка данных'
data = pd.read_excel(path + "/data.xlsx")

data

,name (lat),species,name (ru),description
0,Fringilla coelebs,fricoe,Зяблик,"Величиной примерно с воробья. Наряд самца, ниж..."
1,Motacilla alba,motalb,Трясогузка белая,"Хвост длинный, постоянно покачивающийся вверх ..."
2,Parus major,parmaj,Синица большая,Голова черная с белыми щеками и затылком. Спин...
3,Passer domesticus,pasdom,Воробей домовой,"Окраска оперения коричневато-бурая сверху, бел..."
4,Sturnus vulgaris,stuvul,Скворец обыкновенный,"Оперение черное, с металлическим блеском. Можн..."
5,Turdus merula,turmer,Дрозд чёрный,Оперение тела самца полностью матово-черное. К...


In [4]:
morph = MorphAnalyzer()

def add_pos_tags(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    words = text.split()
    tagged_words = []

    for word in words:
        parsed_word = morph.parse(word)[0]
        tagged_word = f"{word}_{parsed_word.tag.POS}"
        tagged_words.append(tagged_word)
    return ' '.join(tagged_words)

In [5]:
data['pos_description'] = data['description'].apply(add_pos_tags)
data

,name (lat),species,name (ru),description,pos_description
0,Fringilla coelebs,fricoe,Зяблик,"Величиной примерно с воробья. Наряд самца, ниж...",величиной_NOUN примерно_ADVB с_PREP воробья_NO...
1,Motacilla alba,motalb,Трясогузка белая,"Хвост длинный, постоянно покачивающийся вверх ...",хвост_NOUN длинный_ADJF постоянно_ADVB покачив...
2,Parus major,parmaj,Синица большая,Голова черная с белыми щеками и затылком. Спин...,голова_NOUN черная_ADJF с_PREP белыми_ADJF щек...
3,Passer domesticus,pasdom,Воробей домовой,"Окраска оперения коричневато-бурая сверху, бел...",окраска_NOUN оперения_NOUN коричневатобурая_AD...
4,Sturnus vulgaris,stuvul,Скворец обыкновенный,"Оперение черное, с металлическим блеском. Можн...",оперение_NOUN черное_ADJF с_PREP металлическим...
5,Turdus merula,turmer,Дрозд чёрный,Оперение тела самца полностью матово-черное. К...,оперение_NOUN тела_NOUN самца_NOUN полностью_A...


In [9]:
data.to_csv(path + "/bird_text.csv", index=False)

In [11]:
act = pd.read_excel(path + "/bird_action.xlsx")

act.to_csv(path + "/bird_action.csv", index=False)

## Заполнеие графа знаний

In [12]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.4 MB/s eta 0:00:00


In [13]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL
from urllib.parse import quote
import os

In [14]:
graph = Graph()
graph.parse(path + "/BirdsOntology.ttl", format='turtle')
graph

<Graph identifier=N36a5dde1760c4ae2b5edd37f6310bf04 (<class 'rdflib.graph.Graph'>)>

In [15]:
print(graph.serialize(format='ttl'))

@prefix : <http://visualdataweb.org/newOntology/> .
@prefix dc1: <http://purl.org/dc/elements/1.1/#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix s: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Description a owl:DatatypeProperty ;
    s:label "Description"@en ;
    s:domain :Appearance ;
    s:range rdf:PlainLiteral .

:FriCoe a owl:Class ;
    s:label "FriCoe"@en ;
    s:subClassOf :Species .

:MotAlb a owl:Class ;
    s:label "MotAlb"@en ;
    s:subClassOf :Species .

:ParMaj a owl:Class ;
    s:label "ParMaj"@en ;
    s:subClassOf :Species .

:PasDom a owl:Class ;
    s:label "PasDom"@en ;
    s:subClassOf :Species .

:StuVul a owl:Class ;
    s:label "StuVul"@en ;
    s:subClassOf :Species .

:TurMer a owl:Class ;
    s:label "TurMer"@en ;
    s:subClassOf :Species .

:belongsToSpecies a owl:ObjectProperty ;
    s:label "belongsToSpecies"@en ;
    s:domain :Bi

In [16]:
OWL_URI = 'http://www.semanticweb.org/owl/owlapi/turtle#'
NS = Namespace(OWL_URI)

In [17]:
# Classes
bird_class = NS.Bird
voice_class = NS.Voice
action_class = NS.Action
species_class = NS.Species
appearance_class = NS.Appearance

# Species SubClasses
species_classes = {
    "fricoe": NS.FriCoe,
    "motalb": NS.MotAlb,
    "parmaj": NS.ParMaj,
    "pasdom": NS.PasDom,
    "stuvul": NS.StuVul,
    "turmer": NS.TurMer
}

# Object and Data Properties
description_prop = NS.Description
belongs_to_species_prop = NS.belongsToSpecies
has_action_prop = NS.hasAction
has_appearance_prop = NS.hasAppearance
has_voice_prop = NS.hasVoice
is_carrying_prop = NS.isCarrying
is_feeding_prop = NS.isFeeding
is_flying_prop = NS.isFlying
is_sitting_prop = NS.isSitting
name_ru_prop = NS.nameRu
file_name_prop = NS.fileName

In [18]:
# Загрузка данных из data.xlsx
data_df = pd.read_csv(path + "/bird_text.csv")
for index, row in data_df.iterrows():
    bird_instance_uri = URIRef(OWL_URI + row['name (lat)'].replace(' ', '_'))
    species_class_uri = species_classes.get(row['species'])

    # Добавление птицы
    graph.add((bird_instance_uri, RDF.type, bird_class))
    graph.add((bird_instance_uri, belongs_to_species_prop, Literal(row['species'])))
    graph.add((bird_instance_uri, name_ru_prop, Literal(row['name (ru)'], lang='ru')))

    # Добавление внешности
    appearance_instance_uri = URIRef(OWL_URI + row['name (lat)'].replace(' ', '_') + "_appearance")
    graph.add((appearance_instance_uri, RDF.type, appearance_class))
    graph.add((appearance_instance_uri, description_prop, Literal(row['description'])))
    graph.add((bird_instance_uri, has_appearance_prop, appearance_instance_uri))

In [19]:
# Загрузка данных из bird_action.xlsx
actions_df = pd.read_csv(path + "/bird_action.csv")
for index, row in actions_df.iterrows():
    species_name  = row['file'].split('_')[0]
    bird_instance_uri = URIRef(OWL_URI + species_name)

    # Создание действия
    action_instance_uri = URIRef(OWL_URI + species_name + "_action" + str(index))
    graph.add((action_instance_uri, RDF.type, action_class))
    graph.add((action_instance_uri, is_flying_prop, Literal(bool(row['isFlying']))))
    graph.add((action_instance_uri, is_sitting_prop, Literal(bool(row['isSitting']))))
    graph.add((action_instance_uri, is_carrying_prop, Literal(bool(row['isCarrying']))))
    graph.add((action_instance_uri, is_feeding_prop, Literal(bool(row['isFeeding']))))
    graph.add((action_instance_uri, file_name_prop, Literal(row['file'])))

    # Связывание действия с птицей
    graph.add((bird_instance_uri, has_action_prop, action_instance_uri))

In [20]:
# Добавление данных о голосе из файлов в папке birds_audio
audio_files = os.listdir(path + "/birds_audio")
for audio_file in audio_files:
    species_name = os.path.splitext(audio_file)[0].split('_')[0]
    bird_instance_uri = URIRef(OWL_URI + species_name)

    # Создание записи голоса
    voice_instance_uri = URIRef(OWL_URI + species_name + "_voice")
    graph.add((voice_instance_uri, RDF.type, voice_class))
    graph.add((bird_instance_uri, has_voice_prop, voice_instance_uri))

In [21]:
graph.serialize(destination=path+"/UpdatedBirdsOntology.ttl", format='turtle')

<Graph identifier=N36a5dde1760c4ae2b5edd37f6310bf04 (<class 'rdflib.graph.Graph'>)>

In [22]:
!pip install tabulate
from tabulate import tabulate

In [23]:
# SPARQL-запрос
# Получение всех птиц и их описаний
query = """
PREFIX ns: <http://www.semanticweb.org/owl/owlapi/turtle#>

SELECT ?bird ?nameRu ?description WHERE {
    ?bird rdf:type ns:Bird .
    ?bird ns:nameRu ?nameRu .
    ?bird ns:hasAppearance ?appearance .
    ?appearance ns:Description ?description .
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.nameRu), str(row.description)])

# Использование tabulate для красивого вывода
headers = ["Name (RU)", "Description"]
print(tabulate(table, headers, tablefmt="grid"))

+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Name (RU)            | Description                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [24]:
# Получение всех изображений птиц и их действий
query = """
PREFIX ns: <http://www.semanticweb.org/owl/owlapi/turtle#>

SELECT ?file ?isFlying ?isSitting ?isCarrying ?isFeeding WHERE {
    ?action rdf:type ns:Action .
    ?action ns:fileName ?file .
    OPTIONAL { ?action ns:isFlying ?isFlying . }
    OPTIONAL { ?action ns:isSitting ?isSitting . }
    OPTIONAL { ?action ns:isCarrying ?isCarrying . }
    OPTIONAL { ?action ns:isFeeding ?isFeeding . }
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.file), str(row.isFlying), str(row.isSitting), str(row.isCarrying), str(row.isFeeding)])

# Использование tabulate для красивого вывода
headers = ["File", "Is Flying", "Is Sitting", "Is Carrying", "Is Feeding"]
print(tabulate(table, headers, tablefmt="grid"))

+---------------+-------------+--------------+---------------+--------------+
| File          | Is Flying   | Is Sitting   | Is Carrying   | Is Feeding   |
+===============+=============+==============+===============+==============+
| fricoe_0.jfif | false       | true         | false         | false        |
+---------------+-------------+--------------+---------------+--------------+
| fricoe_1.jfif | false       | true         | true          | false        |
+---------------+-------------+--------------+---------------+--------------+
| fricoe_2.jfif | true        | false        | false         | false        |
+---------------+-------------+--------------+---------------+--------------+
| fricoe_3.jfif | true        | false        | false         | false        |
+---------------+-------------+--------------+---------------+--------------+
| fricoe_4.jfif | false       | true         | false         | true         |
+---------------+-------------+--------------+---------------+--

In [25]:
# Получение всех изображений птиц и их действий
query = """
PREFIX ns: <http://www.semanticweb.org/owl/owlapi/turtle#>

SELECT ?bird ?nameRu ?species WHERE {
    ?bird rdf:type ns:Bird .
    ?bird ns:nameRu ?nameRu .
    ?bird ns:belongsToSpecies ?species .
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.nameRu), str(row.species)])

# Использование tabulate для красивого вывода
headers = ["Name Ru", "Species"]
print(tabulate(table, headers, tablefmt="grid"))

+----------------------+-----------+
| Name Ru              | Species   |
+======================+===========+
| Зяблик               | fricoe    |
+----------------------+-----------+
| Трясогузка белая     | motalb    |
+----------------------+-----------+
| Синица большая       | parmaj    |
+----------------------+-----------+
| Воробей домовой      | pasdom    |
+----------------------+-----------+
| Скворец обыкновенный | stuvul    |
+----------------------+-----------+
| Дрозд чёрный         | turmer    |
+----------------------+-----------+
